In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import math

class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    Corresponds to:
    - Equation (1): Linear projection of input features
    - Equation (2): Attention score between nodes i and j using shared weights
    - Equation (3): Aggregation via softmax-normalized attention weights
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
        e = self._prepare_attentional_mechanism_input(Wh)

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        # Wh.shape (N, out_feature)
        # self.a.shape (2 * out_feature, 1)
        # Wh1&2.shape (N, 1)
        # e.shape (N, N)
        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
        # broadcast add
        e = Wh1 + Wh2.T
        return self.leakyrelu(e)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import os

def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def accuracy(output, labels):
    probs = torch.exp(output)
    preds = torch.argmax(probs, dim = 1)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def load_multi_data(folder, att_file, edge_list_name):
    att = pd.read_csv(folder + att_file)
    edge_list = []
    for name in edge_list_name:
        edge_list.append(pd.read_csv(folder + name))

    #get y and x
    labels = np.array(att["SARSCoV2"])
    # b,c,d are age groups
    features = sp.csr_matrix(att[[
       'race', 'birth_sex', 'hiv_aids_age_yrs',
       'trans_categ', 'clusterSize',
       'cd4_group', 'sqnetwork_exp', 'eigen',
       'clustering_coef', 'degree', 'log10vl',
       #'priority_clusters',# need numrical
       'dyad',
       'ncovid',
       'pcd0', 'pcd1', 'pcd2',  'page0', 'page1',
      'ptrans1', 'ptrans2', 'ptrans3', 'ptrans5']])
    #features = normalize_features(features)

    #get adj mat
    adj_list = []
    for edge in edge_list:
        #get row col idx for adj matrix
        row_idx = []
        col_idx = []
        for i in range(edge.shape[0]):
            id_from = edge.iloc[i,0]
            id_to = edge.iloc[i,1]
            if sum(att["uid"] == id_from)>0:
              row_id = att.index[att["uid"] == id_from]
              row_idx.append(row_id[0])

            if sum(att["uid"] == id_to)>0:
              col_id = att.index[att["uid"] == id_to]
              col_idx.append(col_id[0])

        # Construct a weighted adjacency matrix using constant weights (value = 1.0)
        # This is referred to as 'weighted coding' as it enables non-binary edge representation,
        # even though all weights are initially the same.
        adj = sp.coo_matrix((np.ones(edge.shape[0]), (row_idx, col_idx)), shape=(att.shape[0], att.shape[0]), dtype=np.float32)

        #Applies symmetric normalization D^{-1/2}AD^{-1/2} for stability in GNN training
        adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
        #normaliza adj
        adj = normalize_adj(adj + sp.eye(adj.shape[0]))
        adj = torch.FloatTensor(np.array(adj.todense()))
        adj_list.append(adj)

    # Note: The adj_list contains the following adjacency matrices in order:
    # adj_list[0] = P (personal contact network)
    # adj_list[1] = H (household co-residency network)
    # adj_list[2] = C (co-location network)
    # adj_list[3-9] = S_k sublayers (site-type-specific co-location layers)


    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(labels)

    dim = len(labels)
    idx_train = range(1915)
    idx_test = range(1915, dim)

    return adj_list, features, labels, idx_train, idx_test

def load_data(folder, att_file, edge_name):
    att = pd.read_csv(folder + att_file)
    edge_list = pd.read_csv(folder + edge_name)

    #get y and x
    labels = np.array(att["SARSCoV2"])
    # b,c,d are age groups

    features = sp.csr_matrix(att[[
       'race', 'birth_sex', 'hiv_aids_age_yrs',
       'trans_categ', 'clusterSize',
       'cd4_group', 'sqnetwork_exp', 'eigen',
       'clustering_coef', 'degree', 'log10vl',
       #'priority_clusters',# need numrical
       'dyad',
       'ncovid',
       'pcd0', 'pcd1', 'pcd2',  'page0', 'page1',
      'ptrans1', 'ptrans2', 'ptrans3', 'ptrans5']])
    #features = normalize_features(features)

    #get adj mat
    row_idx = []
    col_idx = []
    for i in range(edge_list.shape[0]):
        id_from = edge_list.iloc[i,0]
        id_to = edge_list.iloc[i,1]
        if sum(att["uid"] == id_from)>0:
          row_id = att.index[att["uid"] == id_from]
          row_idx.append(row_id[0])

        if sum(att["uid"] == id_to)>0:
          col_id = att.index[att["uid"] == id_to]
          col_idx.append(col_id[0])


    adj = sp.coo_matrix((np.ones(edge_list.shape[0]), (row_idx, col_idx)), shape=(att.shape[0], att.shape[0]), dtype=np.float32)

    # === Symmetrize the adjacency matrix (A = A ∪ A^T) ===
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    # === Apply row normalization to address differences in network density ===
    # This step corresponds to normalization of P, H, and C layers as described in the paper.
    # These layers are treated as weighted graphs and normalized to stabilize training
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj = torch.FloatTensor(np.array(adj.todense()))

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(labels)

    dim = len(labels)
    idx_train = range(1915)
    idx_test = range(1915, dim)

    return adj, features, labels, idx_train, idx_test

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout

        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = nn.Linear(nhid * nheads, nclass)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        ## Equation (4): Concatenate K multi-head outputs and apply linear projection W^O
        x = F.elu(self.out_att(x))
        return F.log_softmax(x, dim=1)



class FusionGAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix
        self.attentions = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions.append(att_list)
            for k, attention in enumerate(self.attentions):
                self.add_module('adj{}, attention_{}'.format(i, k), attention)

        # Define linear layer for integration with L1 regularization
        self.integration_att = nn.Linear(nhid * nheads, nclass)
        self.fusion = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg = nn.L1Loss(reduction='mean')

    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)
        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):          
            x_i = torch.cat([att(x, adj) for att in self.attentions[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att(x_i))
            output_list.append(x_i)
        output = torch.cat(output_list, dim=1)

        # Apply linear layer for integration with L1 regularization
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion(output.view(output.size(0), -1))
        l1_loss = self.l1_reg(self.fusion.weight, torch.zeros_like(self.fusion.weight))
        return F.log_softmax(output, dim=1), l1_loss



class FusionGAT2(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT2, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix in attention 1
        self.attentions1 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid1, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions1.append(att_list)
            for k, attention in enumerate(self.attentions1):
                self.add_module('adj{}, attention_layer1_{}'.format(i, k), attention)

        # Define linear layer for integration of multihead attention1
        self.integration_att1 = nn.Linear(nhid1 * nheads, nhid1)

        self.attentions2 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list2 = nn.ModuleList([GraphAttentionLayer(nhid1, nhid2, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions2.append(att_list2)
            for k, attention in enumerate(self.attentions2):
                self.add_module('adj{}, attention_layer2_{}'.format(i, k), attention)

        # Define linear layer for integration of multihead attention2
        self.integration_att2 = nn.Linear(nhid2 * nheads, nclass)

        # Fusion layer: dense (fully connected) layer integrating concatenated GAT outputs from all layer with L1 Penalty
        self.fusion_att = nn.Linear(nclass * len(adj_list), nclass)
        # L1 regularization encourages sparsity by penalizing non-zero weights in fusion layers
        self.l1_reg = nn.L1Loss(reduction='mean')

    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)
        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            #attention layer 1
            x_i = torch.cat([att(x, adj) for att in self.attentions1[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att1(x_i))

            # === Equation (7): Fusion of H_concat^(1) → H_fused^(1) ===
            # This is done implicitly after all x_i from layers are collected into output_list

            # === Equation (8): Apply GAT_2 to fused representation H_fused^(1) ===
            #attention layer 2
            x_i = torch.cat([att(x_i, adj) for att in self.attentions2[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att2(x_i))
            output_list.append(x_i)

        output = torch.cat(output_list, dim=1)
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion_att(output.view(output.size(0), -1))
        l1_loss = self.l1_reg(self.fusion_att.weight, torch.zeros_like(self.fusion_att.weight))
        return F.log_softmax(output, dim=1), l1_loss



class FusionGAT3(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, fusion1_dim, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT3, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix
        #att 1
        self.attentions1 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid1, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions1.append(att_list)
            for k, attention in enumerate(self.attentions1):
                self.add_module('adj{}, attention_layer1_{}'.format(i, k), attention)

        # fusion1
        self.integration_att1 = nn.Linear(nhid1 * nheads, fusion1_dim)
        self.fusion_att1 = nn.Linear(fusion1_dim * len(adj_list), fusion1_dim)
        self.l1_reg1 = nn.L1Loss(reduction='mean')

        #att 2
        self.attentions2 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(fusion1_dim, nhid2, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions2.append(att_list)
            for k, attention in enumerate(self.attentions2):
                self.add_module('adj{}, attention_layer2_{}'.format(i, k), attention)

        #fusion2
        self.integration_att2 = nn.Linear(nhid2 * nheads, nclass)
        self.fusion_att2 = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg2 = nn.L1Loss(reduction='mean')


    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)

        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            # Equation (6): Apply multi-head GAT (Layer 1) independently on each adjacency matrix A ∈ {P, H, C, S^(k)}
            # Each attention head computes attention coefficients α_ij over neighbors and aggregates using W^(1)
            # Resulting head outputs are concatenated into H_A^(1)
            x_i = x
            x_i = torch.cat([att(x, adj) for att in self.attentions1[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att1(x_i))
            output_list.append(x_i)
        # Equation (5): Concatenate embeddings from L layers into unified vector c_i    
        output = torch.cat(output_list, dim=1)

        # Apply linear layer for integration with L1 regularization
        # === Equation (9): First L1-regularized fusion layer ===
        # H_fused^(1) = ReLU(Linear_fuse1(H_concat^(1))) with L1 penalty on W_fuse1
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion_att1(output.view(output.size(0), -1))
        l1_loss1 = self.l1_reg1(self.fusion_att1.weight, torch.zeros_like(self.fusion_att1.weight))

        # === Equation (10): GAT_2 per layer, using H_fused^(1) as input ===
        # H_A^(2) = GAT_2(H_fused^(1), A)
        output_list2 = []
        for i, adj in enumerate(adj_list):
            x_i = torch.cat([att(output, adj) for att in self.attentions2[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att2(x_i))
            output_list2.append(x_i)
        output2 = torch.cat(output_list2, dim=1)

        # Apply linear layer for integration with L1 regularization
        # === Equation (11): Second L1-regularized fusion layer ===
        # H_fused^(2) = ReLU(Linear_fuse2(H_concat^(2))) with L1 penalty on W_fuse2
        output2 = F.dropout(output2, self.dropout, training=self.training)
        output2 = self.fusion_att2(output2.view(output.size(0), -1))
        l1_loss2 = self.l1_reg2(self.fusion_att2.weight, torch.zeros_like(self.fusion_att2.weight))

        return F.log_softmax(output2, dim=1), l1_loss1 + l1_loss2

class GCN(nn.Module):
    # Standard 2-layer GCN model implementation based on Kipf & Welling (2016)
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)



# GCN

In [ ]:
import os
import glob
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import precision_recall_curve
import time
import json
from sklearn import metrics

#from load import accuracy, load_multi_data
#from models import FusionGAT3


seed = 72
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output  = model(features, adj_list_tt)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    print('Epoch: {:03d}'.format(epoch+1),
          'loss_train: {:.3f}'.format(loss_train.data.item()),
          'acc_train: {:.3f}'.format(acc_train.data.item()),
          'loss_test: {:.3f}'.format(loss_test.data.item()),
          'acc_test: {:.3f}'.format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          'time: {:.3f}s'.format(time.time() - t))

    return loss_test.data.item(), acc_test, auc_score

def compute_test(model):
    model.eval()
    output = model(features, adj_list_tt)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    # compute false alarm rate and f1 score
    y_pred = np.array([0 if p < 0.5 else 1 for p in y_prob])
    false_alarm_rate = np.mean(y_pred[idx_test.cpu().detach().numpy()] == 1)
    f1score = f1_score(labels[idx_test].cpu().detach().numpy(), y_pred[idx_test.cpu().detach().numpy()])
    precisionn, recalll, _ = precision_recall_curve(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])

    pr_auc = metrics.auc(recalll, precisionn)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          "False Alarm Rate = {:.4f}".format(false_alarm_rate),
          "F1 Score = {:.4f}".format(f1score))
    return loss_test.data.item(), acc_test.data.item(), auc_score, false_alarm_rate, f1score, y_pred, pr_auc


#parameter setting
#

cuda = True
epochs = 500
lrs = [ 0.001]
weight_decays = [ 5e-3]
dropouts = [0.5]
hidden_dims1 = [64]
#hidden_dims2 = [16, 64,128]
#fusion1_dims = [2,4]
#nb_heads = [8, 12]
alpha = 0.2 #Alpha for the leaky_relu
#lambda_l1s = [0.0001, 0.001, 0.01]
patience = 50

#lrs = [0.0005,0.005]
#weight_decays = 0.0005
#dropouts = [0.3]
#hidden_dims1 = [64]
#hidden_dims2 = [64]
#fusion1_dims = [4]
#nb_heads = [8]
#alpha = 0.2 #Alpha for the leaky_relu
#lambda_l1s = [0.0001, 0.001, 0.01]
#patience = 50



# Load data
sv_path = ""
folder = "../data/"
att_name = "att_RR3.csv"
#edge_list_name = ["edge_pp_main.csv", "edge_hh_main.csv", "edge_pv_main_1.csv", "edge_pv_main_2.csv", "edge_pv_main_3.csv"
#, "edge_pv_main_4.csv", "edge_pv_main_5.csv", "edge_pv_main_6.csv", "edge_pv_main_7.csv"]
edge_list_name = "edge_sqRR3.csv"
adj_list, features, labels, idx_train, idx_test = load_data(folder, att_name, edge_list_name)

#adj_list_tensor = torch.stack(adj_list)
features, adj_list_tt, labels = Variable(features), adj_list, Variable(labels)





count = 0
for lr in lrs:
    for dropout in dropouts:
        for hid1 in hidden_dims1:
                        for weight_decay in weight_decays:


                                # Model and optimizer
                                model = GCN(nfeat=features.shape[1],
                                            nhid=hid1,
                                            nclass=int(labels.max()) + 1,
                                            dropout=dropout)

                                optimizer = optim.Adam(model.parameters(),
                                                    lr=lr,
                                                    weight_decay=weight_decay)

                                if cuda:
                                    model.cuda()
                                    features = features.cuda()
                                    adj_list_tt = adj_list_tt.cuda()
                                    labels = labels.cuda()
                                    idx_train = torch.tensor(idx_train).cuda()
                                    idx_test = torch.tensor(idx_test).cuda()


                                # Train model
                                t_total = time.time()
                                bad_counter = 0
                                best_auc = -1
                                best_epoch = 0
                                for epoch in range(epochs):
                                    loss, acc, auc = train(epoch)

                                    torch.save(model.state_dict(), sv_path + '{}.pkl'.format(epoch))
                                    if auc > best_auc:
                                        best_auc = auc
                                        best_epoch = epoch
                                        bad_counter = 0
                                    else:
                                        bad_counter += 1

                                    if bad_counter == patience:
                                        break

                                files = glob.glob(sv_path +'*.pkl')
                                for file in files:
                                    filename = file.split('/')[-1]
                                    epoch_nb = int(filename.split('.')[0])
                                    if epoch_nb != best_epoch:
                                        os.remove(file)

                                print("Optimization Finished!")
                                print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

                                # Restore best model
                                print('Loading {}th epoch'.format(best_epoch))
                                model.load_state_dict(torch.load(sv_path +'{}.pkl'.format(best_epoch)))

                                # Testing
                                test_loss, test_acc, auc, far, f1, y_pred, pr_auc= compute_test(model)
                                hyper_para = {}
                                hyper_para["lr"] = lr
                                hyper_para["weight_decay"] = weight_decay
                                hyper_para["dropout"] = dropout
                                hyper_para["hidden_dim1"] = hid1
                                #hyper_para["hidden_dim2"] = hid2
                                #hyper_para["lambda"] = lambda_l1
                                #hyper_para["fusion1_dim"] = fusion1_dim
                                #hyper_para["nb_heads"] = nb_head
                                #hyper_para["alpha"] = alpha
                                hyper_para["loss"] = test_loss
                                hyper_para["accuracy"] = test_acc
                                hyper_para["auc"] = auc
                                hyper_para["pr_auc"] = pr_auc
                                hyper_para["false_alarm_rate"] = far
                                hyper_para["f1_score"] = f1
                                with open(sv_path +"hyperpara.json", "a+") as fp:
                                    fp.write('\n')
                                    json.dump(hyper_para, fp)

Epoch: 001 loss_train: 1.336 acc_train: 0.919 loss_test: 1.487 acc_test: 0.918 AUC = 0.47 time: 1.009s
Epoch: 002 loss_train: 1.250 acc_train: 0.919 loss_test: 1.392 acc_test: 0.916 AUC = 0.45 time: 0.006s
Epoch: 003 loss_train: 1.221 acc_train: 0.916 loss_test: 1.296 acc_test: 0.916 AUC = 0.43 time: 0.005s
Epoch: 004 loss_train: 1.078 acc_train: 0.912 loss_test: 1.218 acc_test: 0.909 AUC = 0.46 time: 0.006s
Epoch: 005 loss_train: 1.030 acc_train: 0.909 loss_test: 1.113 acc_test: 0.904 AUC = 0.43 time: 0.004s
Epoch: 006 loss_train: 0.949 acc_train: 0.907 loss_test: 0.991 acc_test: 0.917 AUC = 0.50 time: 0.004s
Epoch: 007 loss_train: 0.798 acc_train: 0.905 loss_test: 0.933 acc_test: 0.899 AUC = 0.49 time: 0.004s
Epoch: 008 loss_train: 0.731 acc_train: 0.889 loss_test: 0.949 acc_test: 0.877 AUC = 0.44 time: 0.004s
Epoch: 009 loss_train: 0.738 acc_train: 0.886 loss_test: 0.738 acc_test: 0.885 AUC = 0.52 time: 0.006s
Epoch: 010 loss_train: 0.708 acc_train: 0.844 loss_test: 0.773 acc_test: 

# GAT

In [ ]:
import os
import glob
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import precision_recall_curve
import time
import json
from sklearn import metrics

seed = 72
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output  = model(features, adj_list_tt)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    print('Epoch: {:03d}'.format(epoch+1),
          'loss_train: {:.3f}'.format(loss_train.data.item()),
          'acc_train: {:.3f}'.format(acc_train.data.item()),
          'loss_test: {:.3f}'.format(loss_test.data.item()),
          'acc_test: {:.3f}'.format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          'time: {:.3f}s'.format(time.time() - t))

    return loss_test.data.item(), acc_test, auc_score

def compute_test(model):
    model.eval()
    output = model(features, adj_list_tt)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    # compute false alarm rate and f1 score
    y_pred = np.array([0 if p < 0.5 else 1 for p in y_prob])
    false_alarm_rate = np.mean(y_pred[idx_test.cpu().detach().numpy()] == 1)
    f1score = f1_score(labels[idx_test].cpu().detach().numpy(), y_pred[idx_test.cpu().detach().numpy()])
    precisionn, recalll, _ = precision_recall_curve(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])

    pr_auc = metrics.auc(recalll, precisionn)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          "False Alarm Rate = {:.4f}".format(false_alarm_rate),
          "F1 Score = {:.4f}".format(f1score))
    return loss_test.data.item(), acc_test.data.item(), auc_score, false_alarm_rate, f1score, y_pred, pr_auc

#parameter setting
#

cuda = True
epochs = 500
lrs = [ 0.001]
weight_decays = [5e-5]
dropouts = [0.3]
hidden_dims1 = [64]
#hidden_dims2 = [16, 64,128]
#fusion1_dims = [2,4]
nb_heads = [ 12]
alpha = 0.2 #Alpha for the leaky_relu
#lambda_l1s = [0.0001, 0.001, 0.01]
patience = 50

#lrs = [0.0005,0.005]
#weight_decays = 0.0005
#dropouts = [0.3]
#hidden_dims1 = [64]
#hidden_dims2 = [64]
#fusion1_dims = [4]
#nb_heads = [8]
#alpha = 0.2 #Alpha for the leaky_relu
#lambda_l1s = [0.0001, 0.001, 0.01]
#patience = 50



# Load data
sv_path = ""
folder = "../data/"
att_name = "att_RR3.csv"
#edge_list_name = ["edge_pp_main.csv", "edge_hh_main.csv", "edge_pv_main_1.csv", "edge_pv_main_2.csv", "edge_pv_main_3.csv"
#, "edge_pv_main_4.csv", "edge_pv_main_5.csv", "edge_pv_main_6.csv", "edge_pv_main_7.csv"]
edge_list_name = "edge_sqRR3.csv"
adj_list, features, labels, idx_train, idx_test = load_data(folder, att_name, edge_list_name)

#adj_list_tensor = torch.stack(adj_list)
features, adj_list_tt, labels = Variable(features), adj_list, Variable(labels)





count = 0
for lr in lrs:
    for dropout in dropouts:
        for hid1 in hidden_dims1:
          for nb_head in nb_heads:
                        for weight_decay in weight_decays:


                                # Model and optimizer
                                model = GAT(nfeat=features.shape[1],
                                            nhid=hid1,
                                            nclass=int(labels.max()) + 1,
                                            dropout=dropout,
                                            alpha=alpha,
                                            nheads = nb_head)

                                optimizer = optim.Adam(model.parameters(),
                                                    lr=lr,
                                                    weight_decay=weight_decay)

                                if cuda:
                                    model.cuda()
                                    features = features.cuda()
                                    adj_list_tt = adj_list_tt.cuda()
                                    labels = labels.cuda()
                                    idx_train = torch.tensor(idx_train).cuda()
                                    idx_test = torch.tensor(idx_test).cuda()


                                # Train model
                                t_total = time.time()
                                bad_counter = 0
                                best_auc = -1
                                best_epoch = 0
                                for epoch in range(epochs):
                                    loss, acc, auc = train(epoch)

                                    torch.save(model.state_dict(), sv_path + '{}.pkl'.format(epoch))
                                    if auc > best_auc:
                                        best_auc = auc
                                        best_epoch = epoch
                                        bad_counter = 0
                                    else:
                                        bad_counter += 1

                                    if bad_counter == patience:
                                        break

                                files = glob.glob(sv_path +'*.pkl')
                                for file in files:
                                    filename = file.split('/')[-1]
                                    epoch_nb = int(filename.split('.')[0])
                                    if epoch_nb != best_epoch:
                                        os.remove(file)

                                print("Optimization Finished!")
                                print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

                                # Restore best model
                                print('Loading {}th epoch'.format(best_epoch))
                                model.load_state_dict(torch.load(sv_path +'{}.pkl'.format(best_epoch)))

                                # Testing
                                test_loss, test_acc, auc, far, f1, y_pred, pr_auc= compute_test(model)
                                hyper_para = {}
                                hyper_para["lr"] = lr
                                hyper_para["weight_decay"] = weight_decay
                                hyper_para["dropout"] = dropout
                                hyper_para["hidden_dim1"] = hid1
                                #hyper_para["hidden_dim2"] = hid2
                                #hyper_para["lambda"] = lambda_l1
                                #hyper_para["fusion1_dim"] = fusion1_dim
                                hyper_para["nb_heads"] = nb_head
                                hyper_para["alpha"] = alpha
                                hyper_para["loss"] = test_loss
                                hyper_para["accuracy"] = test_acc
                                hyper_para["auc"] = auc
                                hyper_para["pr_auc"] = pr_auc
                                hyper_para["false_alarm_rate"] = far
                                hyper_para["f1_score"] = f1
                                with open(sv_path +"hyperpara.json", "a+") as fp:
                                    fp.write('\n')
                                    json.dump(hyper_para, fp)

Epoch: 001 loss_train: 1.974 acc_train: 0.374 loss_test: 1.839 acc_test: 0.401 AUC = 0.49 time: 0.195s
Epoch: 002 loss_train: 0.716 acc_train: 0.738 loss_test: 0.682 acc_test: 0.768 AUC = 0.54 time: 0.093s
Epoch: 003 loss_train: 0.580 acc_train: 0.881 loss_test: 0.611 acc_test: 0.890 AUC = 0.44 time: 0.088s
Epoch: 004 loss_train: 0.614 acc_train: 0.913 loss_test: 0.591 acc_test: 0.913 AUC = 0.53 time: 0.070s
Epoch: 005 loss_train: 0.609 acc_train: 0.917 loss_test: 0.625 acc_test: 0.918 AUC = 0.55 time: 0.071s
Epoch: 006 loss_train: 0.585 acc_train: 0.916 loss_test: 0.650 acc_test: 0.918 AUC = 0.50 time: 0.071s
Epoch: 007 loss_train: 0.537 acc_train: 0.918 loss_test: 0.595 acc_test: 0.917 AUC = 0.48 time: 0.069s
Epoch: 008 loss_train: 0.530 acc_train: 0.920 loss_test: 0.537 acc_test: 0.918 AUC = 0.52 time: 0.069s
Epoch: 009 loss_train: 0.467 acc_train: 0.920 loss_test: 0.600 acc_test: 0.920 AUC = 0.41 time: 0.069s
Epoch: 010 loss_train: 0.473 acc_train: 0.918 loss_test: 0.487 acc_test: 

# FGAT


In [ ]:
import os
import glob
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, f1_score

import time
import json

#from load import accuracy, load_multi_data
#from models import FusionGAT3


seed = 72
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output, l1_loss = model(features, adj_list_tt)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train]) + l1_loss*lambda_l1
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    loss_test = F.nll_loss(output[idx_test], labels[idx_test]) + l1_loss*lambda_l1
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    print('Epoch: {:03d}'.format(epoch+1),
          'loss_train: {:.3f}'.format(loss_train.data.item()),
          'acc_train: {:.3f}'.format(acc_train.data.item()),
          'loss_test: {:.3f}'.format(loss_test.data.item()),
          'acc_test: {:.3f}'.format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          'time: {:.3f}s'.format(time.time() - t))

    return loss_test.data.item(), acc_test, auc_score

def compute_test(model):
    model.eval()
    output, l1_loss = model(features, adj_list_tt)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test]) + l1_loss*lambda_l1
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    # compute false alarm rate and f1 score
    y_pred = np.array([0 if p < 0.5 else 1 for p in y_prob])
    false_alarm_rate = np.mean(y_pred[idx_test.cpu().detach().numpy()] == 1)
    f1score = f1_score(labels[idx_test].cpu().detach().numpy(), y_pred[idx_test.cpu().detach().numpy()])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          "False Alarm Rate = {:.4f}".format(false_alarm_rate),
          "F1 Score = {:.4f}".format(f1score))
    return loss_test.data.item(), acc_test.data.item(), auc_score, false_alarm_rate, f1score, y_pred


#parameter setting
#

cuda = True
epochs = 500

lrs = [0.0005]
weight_decays = [5e-5, 5e-4]
dropouts = [0.3]
hidden_dims1 = [64]
hidden_dims2 = [64]
fusion1_dims = [2,4]
nb_heads = [12]
alpha = 0.2 #Alpha for the leaky_relu
lambda_l1s = [0.0001, 0.001, 0.01]
patience = 50

# lrs = [0.0005]
# weight_decays = [5e-5]
# dropouts = [0.3]
# hidden_dims1 = [64]
# hidden_dims2 = [64]
# fusion1_dims = [4]
# nb_heads = [12]
# alpha = 0.2 #Alpha for the leaky_relu
# lambda_l1s = [0.0001]
# patience = 50


# Load data
sv_path = ""
folder = "../data/"
att_name = "att_RR3.csv"

edge_list_name = ["edge_sqRR3.csv", "edge_clRR3.csv"]

adj_list, features, labels, idx_train, idx_test = load_multi_data(folder, att_name, edge_list_name)

adj_list_tensor = torch.stack(adj_list)
features, adj_list_tt, labels = Variable(features), adj_list_tensor, Variable(labels)

count = 0
for lr in lrs:
    for dropout in dropouts:
        for hid1 in hidden_dims1:
            for hid2 in hidden_dims2:
                for fusion1_dim in fusion1_dims:
                    for nb_head in nb_heads:
                        for weight_decay in weight_decays:
                            for lambda_l1 in lambda_l1s:

                                # Model and optimizer
                                model = FusionGAT3(nfeat=features.shape[1],
                                            nhid1=hid1,
                                            nhid2=hid2,
                                            fusion1_dim = fusion1_dim,
                                            nclass=int(labels.max()) + 1,
                                            dropout=dropout,
                                            alpha=alpha,
                                            adj_list= adj_list_tt,
                                            nheads = nb_head)

                                optimizer = optim.Adam(model.parameters(),
                                                    lr=lr,
                                                    weight_decay=weight_decay)

                                if cuda:
                                    model.cuda()
                                    features = features.cuda()
                                    adj_list_tt = adj_list_tt.cuda()
                                    labels = labels.cuda()
                                    idx_train = torch.tensor(idx_train).cuda()
                                    idx_test = torch.tensor(idx_test).cuda()


                                # Train model
                                t_total = time.time()
                                bad_counter = 0
                                best_auc = -1
                                best_epoch = 0
                                for epoch in range(epochs):
                                    loss, acc, auc = train(epoch)

                                    torch.save(model.state_dict(), sv_path + '{}.pkl'.format(epoch))
                                    if auc > best_auc:
                                        best_auc = auc
                                        best_epoch = epoch
                                        bad_counter = 0
                                    else:
                                        bad_counter += 1

                                    if bad_counter == patience:
                                        break

                                files = glob.glob(sv_path +'*.pkl')
                                for file in files:
                                    filename = file.split('/')[-1]
                                    epoch_nb = int(filename.split('.')[0])
                                    if epoch_nb != best_epoch:
                                        os.remove(file)

                                print("Optimization Finished!")
                                print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

                                # Restore best model
                                print('Loading {}th epoch'.format(best_epoch))
                                model.load_state_dict(torch.load(sv_path +'{}.pkl'.format(best_epoch)))

                                # Testing
                                test_loss, test_acc, auc, far, f1, y_pred = compute_test(model)
                                hyper_para = {}
                                hyper_para["lr"] = lr
                                hyper_para["weight_decay"] = weight_decay
                                hyper_para["dropout"] = dropout
                                hyper_para["hidden_dim1"] = hid1
                                hyper_para["hidden_dim2"] = hid2
                                hyper_para["lambda"] = lambda_l1
                                hyper_para["fusion1_dim"] = fusion1_dim
                                hyper_para["nb_heads"] = nb_head
                                hyper_para["alpha"] = alpha
                                hyper_para["loss"] = test_loss
                                hyper_para["accuracy"] = test_acc
                                hyper_para["auc"] = auc
                                hyper_para["false_alarm_rate"] = far
                                hyper_para["f1_score"] = f1
                                with open(sv_path +"hyperpara.json", "a+") as fp:
                                    fp.write('\n')
                                    json.dump(hyper_para, fp)

{"lr": 0.0005, "weight_decay": 5e-05, "dropout": 0.3, "hidden_dim1": 64, "hidden_dim2": 64, "lambda": 0.0001, "fusion1_dim": 4, "nb_heads": 12, "alpha": 0.2, "loss": 0.23118849098682404, "accuracy": 0.9195121951219513, "auc": 0.8472691102001447, "false_alarm_rate": 0.0, "f1_score": 0.0}